### Road Map

In [2]:
# Generate constraint functions jsons which provides all possible numbers grouped by digits (done)
# Permutate blockages and force combinations within cells 
# Force such combinations on a grid
# Weed and continue backtrack cell by cell till final answer is reached

# grid_class.py has not been added in this commit, it will be added after the current puzzle has ended:) My submission is under skrowemalf

### Imports

In [3]:
%reset -f
import json
import time
from handling_restrictions import get_blockages, get_latest_req, split_list, get_possibilities, joiner
from grid_class import specialgrid
from parameters import PARAMS, CONSTRAINTS, MASTER_NUM
from copy import deepcopy as dc

### Permutate blockages 

In [4]:
# Load the constaint functions which provides all possible numbers grouped by digits for this level
# For each possible blockage possibility
    # (new_req)           (list)         Generate latest requirement with (current_grouping_formula, blockage), which is a list of groups + blockages (-1)
    # (fractioned_req)    (list of list) Get fractioned requirement from get_latest_req (so that we can group by digits)
    # (blockage_results)  (list)         Get all possibilities that satisfy the constraint function

all_possible_blockage = get_blockages(MASTER_NUM)
start = time.time()

global_results = []
global_blocks = []
for row_idx in range(MASTER_NUM):
    grouping_formula = PARAMS[row_idx]
    fn = CONSTRAINTS[row_idx]
    row_results = []
    row_blocks = []
    if fn != "": 
        with open('jsons/'+fn, 'r') as f:
            constraint_function = json.load(f)
        for blockages in all_possible_blockage:
            new_req = get_latest_req(grouping_formula, blockages)
            fractioned_req = split_list(new_req)
            blockage_results = get_possibilities(fractioned_req, constraint_function)
            if blockage_results:
                row_results.append( joiner(new_req, blockage_results)  )
                row_blocks.append(blockages)
    if row_blocks == []:
        row_blocks += all_possible_blockage
    global_results.append(row_results)
    global_blocks.append(row_blocks)
    print(f"Row {row_idx}, Possibilities: {len(row_results)}, {time.time()-start:.0f}s, {fn}" )
    start = time.time()

Row 0, Possibilities: 3, 1s, derive_squares.json
Row 1, Possibilities: 19, 2s, palindrome_p1.json
Row 2, Possibilities: 5, 0s, prime_power.json
Row 3, Possibilities: 0, 0s, 
Row 4, Possibilities: 17, 0s, fibonacci.json
Row 5, Possibilities: 5, 1s, derive_squares.json
Row 6, Possibilities: 0, 0s, 
Row 7, Possibilities: 1, 0s, palindrome_23.json
Row 8, Possibilities: 0, 0s, 
Row 9, Possibilities: 0, 0s, 
Row 10, Possibilities: 4, 2s, palindrome_m1.json


In [5]:
global_list = []
for ROW_NUM in range(MASTER_NUM):
    entire_combi_list = []
    r_row = global_results[ROW_NUM]
    for filled_possibility in r_row:
        workspace = [ [] ]
        for element in filled_possibility:
            new_workspace = []
            if isinstance(element, list):
                while workspace:
                    curr_possibility = workspace.pop() 
                    for num in element:
                        new_workspace.append(curr_possibility + list(str(num)))
            else:
                while workspace:
                    curr_possibility = workspace.pop()
                    new_workspace.append(curr_possibility + [element]) 
            workspace = new_workspace
        entire_combi_list += dc(workspace)
    entire_combi_list = [ [int(x) for x in element] for element in entire_combi_list]
    global_list.append(entire_combi_list)
    print(f'Row: {ROW_NUM} Combi length: {len(entire_combi_list)}')


Row: 0 Combi length: 6
Row: 1 Combi length: 38159
Row: 2 Combi length: 116
Row: 3 Combi length: 0
Row: 4 Combi length: 619
Row: 5 Combi length: 30
Row: 6 Combi length: 0
Row: 7 Combi length: 145
Row: 8 Combi length: 0
Row: 9 Combi length: 0
Row: 10 Combi length: 1458


In [6]:
specialgrid().pprint('all')


Params:
    0   1   2   3   4   5   6   7   8   9  10
 0| 0 | 0 | 0 | 4 | 4 | 4 | 8 | 8 | 9 | 9 | 9 | 0   Square
 1| 0 | 3 | 3 | 3 | 4 | 4 | 8 | 9 | 9 | 9 | 10| 1   1 more than Pall
 2| 0 | 3 | 3 | 4 | 4 | 4 | 8 | 9 | 9 | 9 | 10| 2   prime power prime
 3| 0 | 3 | 3 | 4 | 4 | 5 | 5 | 9 | 10| 10| 10| 3   Sum digits 7
 4| 0 | 3 | 4 | 4 | 9 | 9 | 5 | 9 | 10| 11| 10| 4   Fibonacci
 5| 0 | 9 | 9 | 9 | 9 | 9 | 9 | 9 | 11| 11| 12| 5   Square
 6| 2 | 9 | 9 | 9 | 9 | 6 | 6 | 9 | 11| 11| 11| 6   Multiple of 37
 7| 2 | 2 | 1 | 9 | 1 | 6 | 6 | 9 | 9 | 11| 9 | 7   Pall and %23
 8| 2 | 2 | 1 | 1 | 1 | 6 | 6 | 9 | 9 | 9 | 9 | 8   Product ends 1
 9| 2 | 1 | 1 | 2 | 2 | 2 | 6 | 9 | 9 | 9 | 7 | 9   Multiple of 88
10| 2 | 2 | 2 | 2 | 2 | 6 | 6 | 6 | 9 | 9 | 7 | 10  1 less than Pall
    0   1   2   3   4   5   6   7   8   9  10

Grid:
    0   1   2   3   4   5   6   7   8   9  10
 0|   |   |   |   |   |   |   |   |   |   |   | 0   Square
 1|   |   |   |   |   |   |   |   |   |   |   | 1   1 more than Pall

In [7]:
# Get a ranking of all smallest to largest possible combinations lengths, ignoring 0's
rankings = sorted( [ (idx,len(row)) for idx, row in enumerate(global_list)] , key=lambda x: x[1])
threshold = 200
rankings = sorted([ row for row in rankings if threshold > row[1] > 0])

grid_workspace = [ specialgrid() ]
start = time.time()
for idx, combi_counts in rankings:
    if combi_counts == 0: continue
    combination_list = global_list[idx]
    temp_grid_workspace = []
    while grid_workspace:
        curr_grid = grid_workspace.pop()
        for row in combination_list:
            new_grid = dc(curr_grid)
            new_grid.grid[idx] = row
            if new_grid.valid_check():
                temp_grid_workspace.append(new_grid)

    grid_workspace = temp_grid_workspace    
    print(f"Visited idx: {idx}, Expected Combinations: {combi_counts}, Total grids in workspace: {len(grid_workspace)}" )
print(f'Time taken: {time.time()-start:.2f} seconds')


Visited idx: 0, Expected Combinations: 6, Total grids in workspace: 6
Visited idx: 2, Expected Combinations: 116, Total grids in workspace: 1
Visited idx: 5, Expected Combinations: 30, Total grids in workspace: 30
Visited idx: 7, Expected Combinations: 145, Total grids in workspace: 33
Time taken: 34.28 seconds


In [8]:
# Take above combinations, and now forcefully fill in grids via backtracking.

mass_stop = False
workspace = grid_workspace
min_filled_grid = -1
print('(Iteration, workspace length, min cells filled)')
for iteration_x in range(70):
    new_workspace = []
    print(f"({iteration_x}, {len(workspace)}, {min_filled_grid})", end=" ")
    min_filled_grid = MASTER_NUM**2 + 20
    while workspace:
        curr_grid = workspace.pop()
        i, j, possibilities = curr_grid.find_closest_cell()
        if i == -1 and curr_grid.min_filled() == MASTER_NUM**2: 
            print(f"\nFound solution in {iteration_x} iterations")
            mass_stop = True
            break
        for num in possibilities:
            new_grid = dc(curr_grid)
            new_grid.grid[i][j] = num # Fill any number
            new_grid.weed()           # Weed out any invalid possibilities
            if new_grid.valid_check():  
                min_filled_grid = min(min_filled_grid, new_grid.min_filled())
                new_workspace.append(new_grid)
    if mass_stop: break
    workspace = new_workspace
    if len(new_workspace) == 0: 
        print("There are no more grids available in the workspace")
        break

(Iteration, workspace length, min cells filled)
(0, 33, -1) (1, 66, 59) (2, 33, 60) (3, 33, 61) (4, 33, 62) (5, 33, 63) (6, 66, 64) (7, 66, 65) (8, 231, 66) (9, 518, 67) (10, 1498, 68) (11, 1228, 69) (12, 982, 70) (13, 64, 71) (14, 144, 73) (15, 420, 74) (16, 392, 75) (17, 384, 76) (18, 384, 77) (19, 400, 78) (20, 392, 79) (21, 132, 80) (22, 5, 83) (23, 5, 85) (24, 6, 86) (25, 6, 87) (26, 3, 88) (27, 2, 90) (28, 2, 91) (29, 2, 92) (30, 2, 93) (31, 2, 94) (32, 2, 95) (33, 3, 96) (34, 2, 100) (35, 3, 101) (36, 4, 102) (37, 12, 103) (38, 27, 104) (39, 25, 105) (40, 17, 106) (41, 18, 107) (42, 17, 108) (43, 24, 109) (44, 10, 110) (45, 9, 113) (46, 1, 116) (47, 2, 117) (48, 9, 118) (49, 2, 119) 
Found solution in 49 iterations


In [9]:
# curr_grid is the final grid after all the iterations. 
curr_grid.pprint('all')
print(f"Final result, sum of all numbers in the grid = {curr_grid.final_result():,}")


Params:
    0   1   2   3   4   5   6   7   8   9  10
 0| 0 | 0 | 0 | 4 | 4 | 4 | 8 | 8 | 18| 18| 18| 0   Square
 1| 0 | 3 | 3 | 3 | 4 | . | 8 | 18| 18| 18| . | 1   1 more than Pall
 2| 0 | 3 | 3 | 13| . | 15| 8 | 18| 18| 18| 10| 2   prime power prime
 3| 0 | 3 | 3 | . | 20| 5 | 5 | 18| 14| 14| . | 3   Sum digits 7
 4| 0 | 3 | . | 19| 18| 18| . | 18| 14| 11| 16| 4   Fibonacci
 5| 0 | 17| 17| 17| . | 18| 18| 18| 11| 11| 12| 5   Square
 6| 2 | 17| 17| 17| 17| . | 21| 18| 11| 11| 11| 6   Multiple of 37
 7| 2 | 2 | 1 | 17| 1 | 21| 21| . | 9 | 11| 9 | 7   Pall and %23
 8| 2 | 2 | 1 | 1 | 1 | 21| 21| 9 | 9 | 9 | 9 | 8   Product ends 1
 9| . | 1 | 1 | 6 | 6 | . | 21| 9 | 9 | 9 | 7 | 9   Multiple of 88
10| 6 | 6 | 6 | 6 | 6 | 22| . | 23| 9 | 9 | 7 | 10  1 less than Pall
    0   1   2   3   4   5   6   7   8   9  10

Grid:
    0   1   2   3   4   5   6   7   8   9  10
 0| 1 | 1 | 1 | 2 | 2 | 2 | 3 | 3 | 4 | 4 | 4 | 0   Square
 1| 1 | 3 | 3 | 3 | 2 | . | 3 | 4 | 4 | 4 | . | 1   1 more than Pall